##Pips

In [ ]:
import torch

# If there's a GPU available...
if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: Tesla T4


In [ ]:
%%capture
!pip install sentencepiece
!pip install transformers

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install bnaug

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.5 MB 6.2 MB/s 
     |████████████████████████████████| 23.9 MB 83.3 MB/s 
     |████████████████████████████████| 1.0 MB 69.8 MB/s 
  Attempting uninstall: gensim
    Found existing installation: gensim 3.6.0
    Uninstalling gensim-3.6.0:
      Successfully uninstalled gensim-3.6.0
  Attempting uninstall: transformers
    Found existing installation: transformers 4.25.1
    Uninstalling transformers-4.25.1:
      Successfully uninstalled transformers-4.25.1


In [ ]:
!wget https://huggingface.co/sagorsarker/bangla_word2vec/resolve/main/bangla_word2vec_gen4.zip
!wget https://huggingface.co/sagorsarker/bangla-glove-vectors/resolve/main/bn_glove.300d.zip

!unzip bangla_word2vec_gen4.zip
!unzip bn_glove.300d.zip

!rm -rf bangla_word2vec_gen4.zip
!rm -rf bn_glove.300d.zip

--2023-01-02 16:04:54--  https://huggingface.co/sagorsarker/bangla_word2vec/resolve/main/bangla_word2vec_gen4.zip
Resolving huggingface.co (huggingface.co)... 18.235.116.140, 54.144.222.129, 2600:1f18:147f:e800:8e23:1296:56ee:cadc, ...
Connecting to huggingface.co (huggingface.co)|18.235.116.140|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.huggingface.co/repos/f5/12/f51294acf565f8e7f2b231a61a327e6a79fce680615e3bbbc4850f166e928650/b157fe019ddda848f0195240f636ff2dcb63cbe823c84eed77c129e196105fd1?response-content-disposition=attachment%3B%20filename%3D%22bangla_word2vec_gen4.zip%22&Expires=1672934694&Policy=eyJTdGF0ZW1lbnQiOlt7IlJlc291cmNlIjoiaHR0cHM6Ly9jZG4tbGZzLmh1Z2dpbmdmYWNlLmNvL3JlcG9zL2Y1LzEyL2Y1MTI5NGFjZjU2NWY4ZTdmMmIyMzFhNjFhMzI3ZTZhNzlmY2U2ODA2MTVlM2JiYmM0ODUwZjE2NmU5Mjg2NTAvYjE1N2ZlMDE5ZGRkYTg0OGYwMTk1MjQwZjYzNmZmMmRjYjYzY2JlODIzYzg0ZWVkNzdjMTI5ZTE5NjEwNWZkMT9yZXNwb25zZS1jb250ZW50LWRpc3Bvc2l0aW9uPWF0dGFjaG1lbnQlM0IlMjBmaWxlbmFtZS

## Imports

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:

from transformers import *


In [ ]:
# from bnaug.sentence import BackTranslation
# from bnaug.sentence import TextGeneration
from bnaug.sentence import RandomAugmentation
from bnaug import randaug


##function definitions

In [ ]:
def calc_len(sent):
  word_count = len(sent.split())
  return word_count

In [ ]:
def split_sent_daari(str_w):
  x = str_w.rsplit("। ")
  n = len(x)
  if n==1:
    x = str_w.rsplit("।")
    n = len(x)
  if n==1:
    return -1

  return x
    

##Loading data

In [ ]:
path1 = '/content/drive/MyDrive/datasets/Fake-1K.csv'

In [ ]:
df = pd.read_csv(path1)

In [ ]:
split1 = df
df.head(2)

,articleID,domain,date,category,headline,content,label
0,1,channeldhaka.news,2019-03-14T13:34:14+00:00,International,মুরগির হামলায় শেয়াল নিহত,"বাংলায় একটা প্রবাদ আছে, শেয়ালের কাছে মুরগী বর্...",0
1,2,earki.com,"সেম্বর ১৭, ২০১৮",Miscellaneous,বিটিভিতে যেবার আমি ইন্টারভিউ দিতে গেলাম,"BTV থেকে লোকজন আসছে, ইন্টারভিউ নিবে।চারজনের টি...",0


In [ ]:
split1['text'] = split1['headline'] + '। ' + split1['content']
split1 = split1.drop(['headline','content'],axis=1)
split1.head(2)

,articleID,domain,date,category,label,text
0,1,channeldhaka.news,2019-03-14T13:34:14+00:00,International,0,মুরগির হামলায় শেয়াল নিহত। বাংলায় একটা প্রবাদ আ...
1,2,earki.com,"সেম্বর ১৭, ২০১৮",Miscellaneous,0,বিটিভিতে যেবার আমি ইন্টারভিউ দিতে গেলাম। BTV থ...


In [ ]:
split1['text'][0]

'মুরগির হামলায় শেয়াল নিহত। বাংলায় একটা প্রবাদ আছে, শেয়ালের কাছে মুরগী বর্গা দেওয়া। প্রবাদটা এজন্য বলা হয় যে, শিয়ালের কাজই হলো মুরগী খেয়ে ফেলা। শেয়ালের কাছ থেকে মুরগী কখনো নিজেকে রক্ষা করতে পারেনা।তবে ফ্রান্সের এক বিশ্ববিদ্যালয়ের শিক্ষক-শিক্ষার্থীরা জানাচ্ছে, এবার এক আশ্চর্য্যজনক ঘটনা ঘটছে। মুরগীর হামলায় শিয়াল মারা গেছে।ঘটনাটি গত সপ্তাহের, ফ্রান্সের উত্তর-পূর্বাঞ্চলের বুতানিয়া এলাকার একটি কৃষিবিষয়ক বিদ্যালয়ে। ওই বিদ্যালয়ে একটি মুরগির খামার রয়েছে। মুরগির ঘরের এক কোনায় সকালে মৃত একটি শিয়াল পড়ে থাকতে দেখে শিক্ষার্থীরা।৬ হাজার মুরগি রয়েছে খামারটিতে। শিয়ালটির মৃতদেহ যেখানে পাওয়া গেছে, সেখানে তিন হাজার মুরগি ছিল। মুরগিগুলো সারা দিন বাইরে চরে বেড়ায়। সন্ধ্যা হলে নিজ থেকে ঘরে উঠে আসে। সূর্য ডুবে গেলেই দরজা বন্ধ হয়ে যায় স্বয়ংক্রিয়ভাবে।স্থানীয় বন্য প্রাণী বিশেষজ্ঞরা জানিয়েছেন, এই ঘটনায় তাঁরা অবাক। শিয়ালটি বাচ্চা ও অনভিজ্ঞ ছিল। এতগুলো মুরগির সামনে পড়ে সে সম্ভবত ভড়কে গিয়েছিল।Source- BBC Bangla'

In [ ]:
a_sent = ""
for i in range(len(split1)):
  sent = split1['text'][i]
  if(calc_len(sent)>500):
    a_sent = sent
    print(i,' ', a_sent)
    break;


16   যে কারণে ছেলেদের বাল্যবিবাহ দেয়া উচিত !। ছেলের বয়স ২৩, ছেলেকে বিয়ে দিচ্ছেন না, কারণ আপনার ধারনা ছেলে এখনও ছোট। আপনার কাছে ছেলে অবশ্যই ছোট । কিন্তু সে নিজে জানে, সে আসলে কতটা বড় হয়ে গেছে । সে এটা আপনাকে বিস্তারিত বলতে পারে না। বলতে গেলে আপনার চাইতে সেই লজ্জা বেশি পাবে।আপনি বলতে পারেন ছেলে বিয়ে করলে বউকে খাওয়াবে কি? প্রশ্ন হচ্ছে, যে মেয়েটাকে সে এখন বিয়ে করার কথা ভাবছে, সে কি তাহলে এত বছর না খেয়ে ছিল?? বিয়ে যদি আরও পাচ বছর পরও হয়, মেয়ে কি না খেয়ে থাকবে? মেয়ে যদি বিয়ে ছাড়া কোন ভাবেই না খেয়ে না থাকে তাহলে শুধু বিয়েটা পড়ানোর পরেই কেন খাওয়ার চিন্তা আসবে?? মেয়ে তার বাপের কাছে থাকুক, ছেলের যখন সামর্থ্য হবে মেয়েকে নিয়ে আসবে নিজের কাছে। এখন দেখা যাক বাল্যবিবাহ এর অ্যাডভান্টেজ ডিসঅ্যাডভান্টেজ গুলা।অ্যাডভান্টেজঃছেলের জীবনে টেনশন জাস্ট অর্ধেক হয়ে যাবে। তার বিয়ে হয়েছে মানে তার “গার্লফ্রেণ্ডের বিয়ে হয়ে যাবে” রোজ রোজ এই দুশ্চিন্তা করতে হবে না।বিয়ের পরে মানুষ গোছালো হয়। এই বয়সের ছেলেরা অত্যন্ত অগোছালো হয়ে অপেক্ষায় থাকে কেউ একজন এসে তার জীবনটাকে গুছিয়ে দিয়ে যাবে ভ

##Data Augmentation

###loading augmenter and an example

In [ ]:
raug = RandomAugmentation()


In [ ]:
a_sent

'যে কারণে ছেলেদের বাল্যবিবাহ দেয়া উচিত !। ছেলের বয়স ২৩, ছেলেকে বিয়ে দিচ্ছেন না, কারণ আপনার ধারনা ছেলে এখনও ছোট। আপনার কাছে ছেলে অবশ্যই ছোট । কিন্তু সে নিজে জানে, সে আসলে কতটা বড় হয়ে গেছে । সে এটা আপনাকে বিস্তারিত বলতে পারে না। বলতে গেলে আপনার চাইতে সেই লজ্জা বেশি পাবে।আপনি বলতে পারেন ছেলে বিয়ে করলে বউকে খাওয়াবে কি? প্রশ্ন হচ্ছে, যে মেয়েটাকে সে এখন বিয়ে করার কথা ভাবছে, সে কি তাহলে এত বছর না খেয়ে ছিল?? বিয়ে যদি আরও পাচ বছর পরও হয়, মেয়ে কি না খেয়ে থাকবে? মেয়ে যদি বিয়ে ছাড়া কোন ভাবেই না খেয়ে না থাকে তাহলে শুধু বিয়েটা পড়ানোর পরেই কেন খাওয়ার চিন্তা আসবে?? মেয়ে তার বাপের কাছে থাকুক, ছেলের যখন সামর্থ্য হবে মেয়েকে নিয়ে আসবে নিজের কাছে। এখন দেখা যাক বাল্যবিবাহ এর অ্যাডভান্টেজ ডিসঅ্যাডভান্টেজ গুলা।অ্যাডভান্টেজঃছেলের জীবনে টেনশন জাস্ট অর্ধেক হয়ে যাবে। তার বিয়ে হয়েছে মানে তার “গার্লফ্রেণ্ডের বিয়ে হয়ে যাবে” রোজ রোজ এই দুশ্চিন্তা করতে হবে না।বিয়ের পরে মানুষ গোছালো হয়। এই বয়সের ছেলেরা অত্যন্ত অগোছালো হয়ে অপেক্ষায় থাকে কেউ একজন এসে তার জীবনটাকে গুছিয়ে দিয়ে যাবে ভেবে।

In [ ]:
output = randaug.remove_random_word(a_sent)

In [ ]:
output

'যে কারণে ছেলেদের বাল্যবিবাহ দেয়া উচিত !। ছেলের বয়স ২৩, ছেলেকে বিয়ে দিচ্ছেন না, কারণ আপনার ধারনা ছেলে এখনও ছোট। আপনার কাছে ছেলে অবশ্যই ছোট । কিন্তু নিজে জানে, সে আসলে কতটা বড় হয়ে গেছে । সে এটা আপনাকে বিস্তারিত বলতে পারে না। বলতে গেলে আপনার চাইতে সেই লজ্জা বেশি পাবে।আপনি বলতে পারেন ছেলে বিয়ে করলে বউকে খাওয়াবে কি? প্রশ্ন হচ্ছে, যে মেয়েটাকে সে এখন বিয়ে করার কথা ভাবছে, সে কি তাহলে এত বছর না খেয়ে ছিল?? বিয়ে যদি আরও পাচ বছর পরও হয়, মেয়ে কি না খেয়ে থাকবে? মেয়ে যদি বিয়ে ছাড়া কোন ভাবেই না খেয়ে না থাকে তাহলে শুধু বিয়েটা পড়ানোর পরেই কেন খাওয়ার চিন্তা আসবে?? মেয়ে তার বাপের কাছে থাকুক, ছেলের যখন সামর্থ্য হবে মেয়েকে নিয়ে আসবে নিজের কাছে। এখন দেখা যাক বাল্যবিবাহ এর অ্যাডভান্টেজ ডিসঅ্যাডভান্টেজ গুলা।অ্যাডভান্টেজঃছেলের জীবনে টেনশন জাস্ট অর্ধেক হয়ে যাবে। তার বিয়ে হয়েছে মানে তার “গার্লফ্রেণ্ডের বিয়ে হয়ে যাবে” রোজ রোজ এই দুশ্চিন্তা করতে হবে না।বিয়ের পরে মানুষ গোছালো হয়। এই বয়সের ছেলেরা অত্যন্ত অগোছালো হয়ে অপেক্ষায় থাকে কেউ একজন এসে তার জীবনটাকে গুছিয়ে দিয়ে যাবে ভেবে। এক

In [ ]:
l_split_sents = split_sent_daari(a_sent)

In [ ]:
l_split_sents

['যে কারণে ছেলেদের বাল্যবিবাহ দেয়া উচিত !',
 'ছেলের বয়স ২৩, ছেলেকে বিয়ে দিচ্ছেন না, কারণ আপনার ধারনা ছেলে এখনও ছোট',
 'আপনার কাছে ছেলে অবশ্যই ছোট ',
 'কিন্তু সে নিজে জানে, সে আসলে কতটা বড় হয়ে গেছে ',
 'সে এটা আপনাকে বিস্তারিত বলতে পারে না',
 'বলতে গেলে আপনার চাইতে সেই লজ্জা বেশি পাবে।আপনি বলতে পারেন ছেলে বিয়ে করলে বউকে খাওয়াবে কি? প্রশ্ন হচ্ছে, যে মেয়েটাকে সে এখন বিয়ে করার কথা ভাবছে, সে কি তাহলে এত বছর না খেয়ে ছিল?? বিয়ে যদি আরও পাচ বছর পরও হয়, মেয়ে কি না খেয়ে থাকবে? মেয়ে যদি বিয়ে ছাড়া কোন ভাবেই না খেয়ে না থাকে তাহলে শুধু বিয়েটা পড়ানোর পরেই কেন খাওয়ার চিন্তা আসবে?? মেয়ে তার বাপের কাছে থাকুক, ছেলের যখন সামর্থ্য হবে মেয়েকে নিয়ে আসবে নিজের কাছে',
 'এখন দেখা যাক বাল্যবিবাহ এর অ্যাডভান্টেজ ডিসঅ্যাডভান্টেজ গুলা।অ্যাডভান্টেজঃছেলের জীবনে টেনশন জাস্ট অর্ধেক হয়ে যাবে',
 'তার বিয়ে হয়েছে মানে তার “গার্লফ্রেণ্ডের বিয়ে হয়ে যাবে” রোজ রোজ এই দুশ্চিন্তা করতে হবে না।বিয়ের পরে মানুষ গোছালো হয়',
 'এই বয়সের ছেলেরা অত্যন্ত অগোছালো হয়ে অপেক্ষায় থাকে কেউ একজন এসে তার জীবনটাকে

In [ ]:
#individually backtranslate each sentences
for i in range(len(l_split_sents)):
  l_split_sents[i] = randaug.remove_random_word(l_split_sents[i])


In [ ]:
l_split_sents

['কারণে ছেলেদের বাল্যবিবাহ দেয়া উচিত !',
 'ছেলের বয়স ২৩, ছেলেকে বিয়ে দিচ্ছেন না, কারণ ধারনা ছেলে এখনও ছোট',
 'আপনার কাছে ছেলে ছোট',
 'কিন্তু সে নিজে জানে, সে আসলে বড় হয়ে গেছে',
 'এটা আপনাকে বিস্তারিত বলতে পারে না',
 'বলতে গেলে আপনার চাইতে সেই লজ্জা বেশি পাবে।আপনি বলতে পারেন ছেলে বিয়ে করলে বউকে খাওয়াবে কি? প্রশ্ন হচ্ছে, যে মেয়েটাকে সে এখন বিয়ে করার কথা ভাবছে, সে কি তাহলে এত বছর না খেয়ে ছিল?? বিয়ে আরও পাচ বছর পরও হয়, মেয়ে কি না খেয়ে থাকবে? মেয়ে যদি বিয়ে ছাড়া কোন ভাবেই না খেয়ে না থাকে তাহলে শুধু বিয়েটা পড়ানোর পরেই কেন খাওয়ার চিন্তা আসবে?? মেয়ে তার বাপের কাছে থাকুক, ছেলের যখন সামর্থ্য হবে মেয়েকে নিয়ে আসবে নিজের কাছে',
 'এখন দেখা যাক বাল্যবিবাহ এর অ্যাডভান্টেজ ডিসঅ্যাডভান্টেজ গুলা।অ্যাডভান্টেজঃছেলের জীবনে টেনশন জাস্ট অর্ধেক যাবে',
 'তার বিয়ে হয়েছে মানে তার “গার্লফ্রেণ্ডের বিয়ে হয়ে রোজ রোজ এই দুশ্চিন্তা করতে হবে না।বিয়ের পরে মানুষ গোছালো হয়',
 'এই বয়সের ছেলেরা অত্যন্ত অগোছালো হয়ে অপেক্ষায় থাকে কেউ একজন এসে জীবনটাকে গুছিয়ে দিয়ে যাবে ভেবে',
 'একজন স্ত্রী ছাড়

In [ ]:
#original sentence
a_sent

'যে কারণে ছেলেদের বাল্যবিবাহ দেয়া উচিত !। ছেলের বয়স ২৩, ছেলেকে বিয়ে দিচ্ছেন না, কারণ আপনার ধারনা ছেলে এখনও ছোট। আপনার কাছে ছেলে অবশ্যই ছোট । কিন্তু সে নিজে জানে, সে আসলে কতটা বড় হয়ে গেছে । সে এটা আপনাকে বিস্তারিত বলতে পারে না। বলতে গেলে আপনার চাইতে সেই লজ্জা বেশি পাবে।আপনি বলতে পারেন ছেলে বিয়ে করলে বউকে খাওয়াবে কি? প্রশ্ন হচ্ছে, যে মেয়েটাকে সে এখন বিয়ে করার কথা ভাবছে, সে কি তাহলে এত বছর না খেয়ে ছিল?? বিয়ে যদি আরও পাচ বছর পরও হয়, মেয়ে কি না খেয়ে থাকবে? মেয়ে যদি বিয়ে ছাড়া কোন ভাবেই না খেয়ে না থাকে তাহলে শুধু বিয়েটা পড়ানোর পরেই কেন খাওয়ার চিন্তা আসবে?? মেয়ে তার বাপের কাছে থাকুক, ছেলের যখন সামর্থ্য হবে মেয়েকে নিয়ে আসবে নিজের কাছে। এখন দেখা যাক বাল্যবিবাহ এর অ্যাডভান্টেজ ডিসঅ্যাডভান্টেজ গুলা।অ্যাডভান্টেজঃছেলের জীবনে টেনশন জাস্ট অর্ধেক হয়ে যাবে। তার বিয়ে হয়েছে মানে তার “গার্লফ্রেণ্ডের বিয়ে হয়ে যাবে” রোজ রোজ এই দুশ্চিন্তা করতে হবে না।বিয়ের পরে মানুষ গোছালো হয়। এই বয়সের ছেলেরা অত্যন্ত অগোছালো হয়ে অপেক্ষায় থাকে কেউ একজন এসে তার জীবনটাকে গুছিয়ে দিয়ে যাবে ভেবে।

In [ ]:
#join all sentences to make one single string
result = ''.join(''.join(l) for l in l_split_sents) #nested join
result

'কারণে ছেলেদের বাল্যবিবাহ দেয়া উচিত !ছেলের বয়স ২৩, ছেলেকে বিয়ে দিচ্ছেন না, কারণ ধারনা ছেলে এখনও ছোটআপনার কাছে ছেলে ছোটকিন্তু সে নিজে জানে, সে আসলে বড় হয়ে গেছেএটা আপনাকে বিস্তারিত বলতে পারে নাবলতে গেলে আপনার চাইতে সেই লজ্জা বেশি পাবে।আপনি বলতে পারেন ছেলে বিয়ে করলে বউকে খাওয়াবে কি? প্রশ্ন হচ্ছে, যে মেয়েটাকে সে এখন বিয়ে করার কথা ভাবছে, সে কি তাহলে এত বছর না খেয়ে ছিল?? বিয়ে আরও পাচ বছর পরও হয়, মেয়ে কি না খেয়ে থাকবে? মেয়ে যদি বিয়ে ছাড়া কোন ভাবেই না খেয়ে না থাকে তাহলে শুধু বিয়েটা পড়ানোর পরেই কেন খাওয়ার চিন্তা আসবে?? মেয়ে তার বাপের কাছে থাকুক, ছেলের যখন সামর্থ্য হবে মেয়েকে নিয়ে আসবে নিজের কাছেএখন দেখা যাক বাল্যবিবাহ এর অ্যাডভান্টেজ ডিসঅ্যাডভান্টেজ গুলা।অ্যাডভান্টেজঃছেলের জীবনে টেনশন জাস্ট অর্ধেক যাবেতার বিয়ে হয়েছে মানে তার “গার্লফ্রেণ্ডের বিয়ে হয়ে রোজ রোজ এই দুশ্চিন্তা করতে হবে না।বিয়ের পরে মানুষ গোছালো হয়এই বয়সের ছেলেরা অত্যন্ত অগোছালো হয়ে অপেক্ষায় থাকে কেউ একজন এসে জীবনটাকে গুছিয়ে দিয়ে যাবে ভেবেএকজন স্ত্রী ছাড়া কোন গার্লফ্রেণ্ডের এটা সম্ভব না।ডেটিংয়ের পে

###batch augment whole DS

In [ ]:
split1.head()

,articleID,domain,date,category,label,text
0,1,channeldhaka.news,2019-03-14T13:34:14+00:00,International,0,মুরগির হামলায় শেয়াল নিহত। বাংলায় একটা প্রবাদ আ...
1,2,earki.com,"সেম্বর ১৭, ২০১৮",Miscellaneous,0,বিটিভিতে যেবার আমি ইন্টারভিউ দিতে গেলাম। BTV থ...
2,3,earki.com,"২০:৩৯, জানুয়ারি ১৪, ২০১৯",Miscellaneous,0,বিদেশ থেকে উন্নতমানের বিরোধীদল আমদানি করার পরা...
3,4,channeldhaka.news,2018-06-30T15:56:47+00:00,Sports,0,অবসর নেয়ার ঘোষণা দিলেন মেসি !। রাশিয়া বিশ্বকাপ...
4,5,motikontho.wordpress.com,2013-03-05T21:55:45+00:00,Miscellaneous,0,"মাদারফাকার নহে, ব্রাদারফাকার: সাকা | দৈনিক মতি..."


In [ ]:
len(split1)

1299

In [ ]:
for i in range(20,60):
  if(1300%i)==0:
    print(i)

20
25
26
50
52


In [ ]:
start = 0
end = len(split1)
#use start and end variables to control how much of your DS you want to augment. The output shown after this section 
#shows how much has been successfully converted and saved in you gdrive. 

for i in range(start, end):
  sent = split1['text'][i]

  #on every 25th row, making a save
  if(i%25==0):
    split1.to_csv('/content/drive/MyDrive/datasets/fake_aug_bt.csv', index=False) #bt = backtranslation
    print(i)
  
  #splitting the sentence according to stop words
  split_sents = split_sent_daari(a_sent)
  if(split_sents==-1):
    print(i, 'th row has error. Has not been split properly')
    continue
  for i in range(len(split_sents)):
    split_sents[i] = bt.get_augmented_sentences(split_sents[i])
  
  result = ''.join(''.join(l) for l in l_split_sents) #nested join

  split1.at[i, 'text'] = result

  
  


0


In [ ]:
#done. save all.
split1.to_csv('/content/drive/MyDrive/datasets/fake_aug_bt.csv', index=False) #bt = backtranslation